In [1]:
import tensorflow as tf
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
df= pd.read_csv('Data/train.csv')
all_id = np.array(df['Id'])
IMG_SIZE = 128
colours = 3

def get_images_from_id(data_id, img_size=128, colours=3):
    pictures = np.zeros((len(data_id), IMG_SIZE, IMG_SIZE, colours))
    for i, single_id in tqdm.tqdm(enumerate(data_id)):
        img = cv2.cvtColor(cv2.imread(f"Data/train/{single_id}.jpg"), cv2.COLOR_BGR2RGB)
        new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        pictures[i] = new_img/255
    return pictures

In [3]:
y = df['Pawpularity']
X = get_images_from_id(df['Id'], img_size=128, colours=3)

9912it [01:18, 127.02it/s]


In [4]:
import sys
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import numpy as np
import random
import tensorflow as tf
import pickle
import time
import copy


class Genotype:
    def __init__(self, filters_numbers, filters_sizes, activation_f, random_state=1410):
        self.__filters_numbers = filters_numbers
        self.__filters_sizes = filters_sizes
        self.__activation_f = activation_f
        self.__random_state = random_state


class MyGA:
    def __init__(self, max_generations=5, population_size=30, mutation_prob=0.1, stop_rate=1.1,
                 filters_numbers=[8, 16, 32, 64, 128],
                 filters_sizes=[5, 7, 9], activation_f=['selu', 'tanh'], random_state=1410):
        self.__max_generations = max_generations
        self.__population_size = population_size
        self.__mutation_prob = mutation_prob
        self.__filters_numbers = filters_numbers
        self.__filters_sizes = filters_sizes
        self.__activation_f = activation_f
        self.__add_sth = [0, 1]
        self.__random_state = random_state
        self.__hiperparam_number = 6
        self.__phase_number = 0
        self.__init_population()
        self.__known_genotype = []
        self.__known_genotype_value = []
        self.stop_rate = stop_rate

    def __init_population(self):
        if self.__phase_number == 0:
            self.__population = np.zeros((self.__population_size, self.__hiperparam_number)).astype(int)
            self.__genotype = [self.__activation_f, self.__filters_numbers, self.__filters_sizes, self.__add_sth,
                               self.__filters_numbers, self.__filters_sizes]
            # iteracja po każdym osobniku
            for i in range(self.__population.shape[0]):
                # iteracja po genach genotypu
                for j in range(len(self.__genotype)):
                    #  losowanie liczby z przedziału odpowiadającego ilości kombinaci w genie
                    self.__population[i, j] = random.randint(0, len(self.__genotype[j]) - 1)
        # każda kolejna faza od 1 włącznie
        else:
            # dodanie zer na wartości nowych genów
            gen_to_add = np.zeros((self.__population_size, self.__hiperparam_number)).astype(int)
            # inicjalizacja całej populacji najlepszym osobnikiem
            self.__population = [self.__local_best_individual] * self.__population_size
            # dodanie do genotypu najlepszego osobnika miejsc na nową cześć genu
            self.__population = np.concatenate((self.__population, gen_to_add), axis=1)
            # rozszeżenie genotypu o nowy człon
            self.__genotype.extend([self.__filters_numbers, self.__filters_sizes, self.__add_sth, self.__add_sth,
                                    self.__filters_numbers, self.__filters_sizes])
            for i in range(self.__population.shape[0]):
                # inicjalizacja nowego genu (od końca)
                for j in range(self.__hiperparam_number):
                    max_range = len(self.__genotype[-(j + 1)]) - 1
                    self.__population[i, -(j + 1)] = random.randint(0, max_range)

    def print_individual_info(self, individual_id):
        for i, feature in enumerate(self.__population[individual_id]):
            print(self.__genotype[i][feature])

    def fit(self, X_train, y_train, validation_data):
        self.__X_train = X_train
        self.__y_train = y_train
        self.__X_valid, self.__y_valid = validation_data
        # inicjalizacja wartości funkcji celu wszytskich osobnikiów oraz zwrócenie modeli
        self.__goal_values, self.__models = self.goal_functions()
        # znalezienie najlepszego osobnika
        self.__local_best_individual = copy.deepcopy(self.__population)[np.argmin(self.__goal_values)]
        self.best_params = copy.deepcopy(self.__local_best_individual)
        self.__local_best_model = self.__models[np.argmin(self.__goal_values)]
        self.best_estimator = self.__local_best_model
        self.__local_best_goal = min(self.__goal_values)
        self.best_score = self.__local_best_goal
        new_population = np.zeros((self.__population_size, self.__hiperparam_number)).astype(int)
        self.__phase_number = 0
        # kojejne pokolenia tworzą się do póki najlepsza wartośc w nowym pokoleniu jest większa lub równa globalnie
        # najlepszej wartości pomnożonej przez zadany współczynnik
        while self.stop_rate * self.best_score >= self.__local_best_goal:
            if self.__phase_number != 0:
                # aby jako najlepsze rozwiązanie w pokoleniu przyjść najlepszą wartość z pierwszego krzyżowania
                self.__local_best_goal = 9999999
                self.__init_population()
                new_population = np.zeros(
                    (self.__population_size, self.__hiperparam_number * (self.__phase_number + 1))) \
                    .astype(int)
            for j in range(self.__max_generations):
                for i in range(self.__population_size):
                    mum_id = self.rulette(self.__goal_values)
                    # spełnienie warunki, że mum_id != dad_id
                    dad_id = self.rulette(np.delete(self.__goal_values, mum_id))
                    new_population[i] = self.crossover(mum_id, dad_id)
                self.__population = new_population
                self.__goal_values, self.__models = self.goal_functions()
                # sprawdzenie czy pokolenie dało lepszą wartośc niż najlepsza w fazie
                if min(self.__goal_values) < self.__local_best_goal:
                    self.__local_best_goal = min(self.__goal_values)
                    self.__local_best_individual = copy.deepcopy(self.__population)[np.argmin(self.__goal_values)]
                    self.__local_best_model = self.__models[np.argmin(self.__goal_values)]
                print('***************')
                print(f'Dla generacji {j+1} mse: {self.__local_best_goal}')
                print(self.__local_best_individual)
                print('***************')
            self.__phase_number += 1
            # sprawdzenie czy w fazie uzyskano lepsza wartośc niż globalne maksimum
            if self.__local_best_goal < self.best_score:
                self.best_score = self.__local_best_goal
                self.best_params = copy.deepcopy(self.__local_best_individual)
                self.best_estimator = self.__local_best_model
            self.__known_genotype = []
            self.__known_genotype_value = []
            print("NASTĘPNA EPOKA")
            print('---------------------------------------------------------')
            print(f'Best local value: {self.__local_best_goal}')
            print(f'Best local {self.__local_best_individual}')
            print(f'Best model: {self.__local_best_model.summary()}')
            print('------------')
            print(f'Best global value: {self.best_score}')
            print(f'Best global  {self.best_params}')
            print(f'Best model: {self.best_estimator.summary()}')
            print('---------------------------------------------------------')

    def rulette(self, goal_values):
        rev_goal_values = []
        for goal_value in goal_values:
            if goal_value == 0:
                rev_goal_values.append(0)
            else:
                rev_goal_values.append(1/goal_value)
        all_circle = sum(rev_goal_values)
        drawn_value = random.uniform(0, all_circle)
        for i in range(len(goal_values)):
            if drawn_value <= sum(rev_goal_values[:(i + 1)]):  # (i+1) ponieważ dopiero x[:0] pusta tablice
                return i
            
    

    def crossover(self, mum_id, dad_id):
        children = np.zeros(len(self.__genotype)).astype(int)
        for i in range(self.__hiperparam_number * (self.__phase_number + 1)):
            if random.randint(0, 1):
                children[i] = self.__population[mum_id, i]
            else:
                children[i] = self.__population[dad_id, i]
            if random.uniform(0, 1) < self.__mutation_prob:
                children[i] = random.randint(0, len(self.__genotype[i]) - 1)
        return children

    def goal_functions(self):
        results = np.zeros(self.__population_size)
        models = np.empty(self.__population_size, dtype=object)
        for i, individual in enumerate(self.__population):
            # print(individual)
            # print(self.__population)
            if list(individual) in self.__known_genotype:
                results[i] = self.__known_genotype_value[self.__known_genotype.index(list(individual))]
            else:
                activation_f = self.__activation_f[individual[0]]
                if self.__phase_number == 0:
                    # dla CIFAR-10 32x32x3
                    input_layer = tf.keras.Input((self.__X_train.shape[1], self.__X_train.shape[2],
                                                  self.__X_train.shape[3]))
                    convA = tf.keras.layers.Conv2D(
                        self.__filters_numbers[individual[1]],
                        self.__filters_sizes[individual[2]],
                        activation=activation_f,
                        padding='same')(input_layer)

                    if individual[3]:
                        maxpool2D = tf.keras.layers.MaxPooling2D(2, padding='same')(convA)
                        convB = tf.keras.layers.Conv2D(
                            self.__filters_numbers[individual[4]],
                            self.__filters_sizes[individual[5]],
                            activation=activation_f,
                            padding='same')(maxpool2D)
                    else:
                        convB = tf.keras.layers.Conv2D(
                            self.__filters_numbers[individual[4]],
                            self.__filters_sizes[individual[5]],
                            activation=activation_f,
                            padding='same')(convA)
                    flatten = tf.keras.layers.GlobalMaxPool2D()(convB)
                    output_layer = tf.keras.layers.Dense(1)(flatten)

                    model = tf.keras.Model(inputs=[input_layer], outputs=[output_layer])
                else:
                    # dodanie nowych warstw
                    # warstwa wejściowa
                    model_clone = tf.keras.models.clone_model(self.__local_best_model)
                    lay = model_clone.layers[0].output
                    # dodanie wszytskich (z wyjątkiem ostatniej i przedostatniej) warstwy najlepszego osobnika
                    # w ostatnim pokoleniu modelu
                    for j in range(1, len(model_clone.layers) - 2):
                        lay = model_clone.layers[j](lay)

                    # self.__hiperparam_number*self.__generation_number dla drugiej generacji = 6*1, więc cechy
                    # genotypu wpisywane są od 6 elementu, który stanowi pierwszy nowe element w genotypie
                    filter_number_convA = self.__filters_numbers[
                        individual[self.__hiperparam_number * self.__phase_number]]
                    window_size_convA = self.__filters_sizes[
                        individual[self.__hiperparam_number * self.__phase_number + 1]]
                    add_pooling = self.__add_sth[individual[self.__hiperparam_number * self.__phase_number + 2]]
                    add_convB = self.__add_sth[individual[self.__hiperparam_number * self.__phase_number + 3]]
                    filter_number_convB = self.__filters_numbers[
                        individual[self.__hiperparam_number * self.__phase_number + 4]]
                    window_size_convB = self.__filters_sizes[
                        individual[self.__hiperparam_number * self.__phase_number + 5]]

                    lay = tf.keras.layers.Conv2D(filter_number_convA, window_size_convA, activation=activation_f,
                                                 padding='same')(lay)
                    if add_pooling:
                        lay = tf.keras.layers.MaxPooling2D(2, padding='same')(lay)
                    if add_convB:
                        lay = tf.keras.layers.Conv2D(filter_number_convB, window_size_convB, activation=activation_f,
                                                     padding='same')(lay)
                    lay = tf.keras.layers.GlobalMaxPool2D()(lay)
                    lay = tf.keras.layers.Dense(1)(lay)
                    model = tf.keras.Model(model_clone.layers[0].input, lay)

                    # transfer wag z najlepszego modelu, transfer wag odbywa się bez transferowania wag z dwóch
                    # ostatnich warstw (GlobalMaxPooling i Dense(10))
                    for j in range(len(self.__local_best_model.layers) - 2):
                        model.layers[j].set_weights(self.__local_best_model.layers[j].get_weights())

                model.compile(loss=tf.keras.losses.MeanSquaredError(),
                              optimizer=tf.keras.optimizers.Adam(1e-3),
                              metrics=["mse"])
                # (model.summary())
                model.fit(self.__X_train, self.__y_train, epochs=5, verbose=0)
                # print(model.summary())
                y_pred = model.predict(self.__X_valid)
                results[i] = mean_squared_error(self.__y_valid, y_pred)
                models[i] = model
                self.__known_genotype.append(list(individual))
                self.__known_genotype_value.append(results[i])
        return results, models

    def get_best_model(self, more_epochs=False):
        if more_epochs == False:
            return self.best_estimator
        else:
            checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
            early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
            self.best_estimator.fit(self.__X_train, self.__y_train, epochs=300,
                                    validation_data=(self.__X_valid, self.__y_valid),
                                    callbacks=[checkpoint_cb, early_stopping_cb])
            return tf.keras.models.load_model('best_model.h5')

In [ ]:
model = MyGA(population_size=10, max_generations=3)
model.fit(X[:7000], y[:7000], (X[7000:], y[7000:]))
best_model = model.get_best_model()
# print(best_model.get_weights())
best_model.save('model.h5')


***************
Dla generacji 1 mse: 407.13214179923085
[1 4 1 0 3 1]
***************
***************
Dla generacji 2 mse: 407.13214179923085
[1 4 1 0 3 1]
***************
***************
Dla generacji 3 mse: 407.13214179923085
[1 4 1 0 3 1]
***************
NASTĘPNA EPOKA
---------------------------------------------------------
Best local value: 407.13214179923085
Best local [1 4 1 0 3 1]
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 128)     18944     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      401472    
_________________________________________________________________
global_max_pooling2d (Global (None, 64)                0        

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\BartoszOlszewski(243\miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\BARTOS~1\AppData\Local\Temp/ipykernel_9764/3845357769.py", line 2, in <module>
    model.fit(X[:7000], y[:7000], (X[7000:], y[7000:]))
  File "C:\Users\BARTOS~1\AppData\Local\Temp/ipykernel_9764/3921964860.py", line 103, in fit
    self.__goal_values, self.__models = self.goal_functions()
  File "C:\Users\BARTOS~1\AppData\Local\Temp/ipykernel_9764/3921964860.py", line 238, in goal_functions
    model.fit(self.__X_train, self.__y_train, epochs=5, verbose=0)
  File "C:\Users\BartoszOlszewski(243\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\training.py", line 819, in fit
    use_multiprocessing=use_multiprocessing)
  File "C:\Users\BartoszOlszewski(243\miniconda3\envs\tensorflow\lib\site-packages\tensorflow_cor

In [5]:
import random
def rulette(goal_values):
    rev_goal_values = []
    for goal_value in goal_values:
        if goal_value == 0:
            rev_goal_values.append(0)
        else:
            rev_goal_values.append(1/goal_value)
    all_circle = sum(rev_goal_values)
    drawn_value = random.uniform(0, all_circle)
    for i in range(len(goal_values)):
        if drawn_value <= sum(rev_goal_values[:(i + 1)]):  # (i+1) ponieważ dopiero x[:0] pusta tablice
            return i
        

g = [5, 10, 20, 30, 40, 60, 100]
for i in range(100):
    print(rulette(g))

1
0
0
5
0
1
6
2
3
2
1
0
3
2
1
5
4
0
0
2
0
1
0
3
0
3
0
2
2
1
0
1
2
1
1
2
0
0
5
0
0
1
0
0
0
0
0
3
0
4
2
0
1
0
0
0
1
2
4
2
3
4
1
0
2
0
3
1
2
0
5
0
1
0
6
1
1
2
2
1
0
1
0
0
0
2
5
1
2
0
0
0
0
ERROR! Session/line number was not unique in database. History logging moved to new session 801
0
4
0
2
1
1
0


In [6]:
def roulette(f_goals_array, number_to_generate, rand_gen):
    while(True):
        n = len(f_goals_array)
        f_goals_reversed_values = []
        result = []
        for i in range(n):
            if f_goals_array == 0:
                f_goals_reversed_values.append(0)
            else:
                f_goals_reversed_values.append(1/f_goals_array[i])
        full_circle = sum(f_goals_reversed_values)
    
        rand_gen.nextFloat(0, 1)
        for j in range(number_to_generate):
            drawn_value = rand_gen.nextFloat(0, full_circle)
            present_part_of_circle = 0
            for i, singe_part in enumerate(f_goals_reversed_values):
                present_part_of_circle += singe_part
                if present_part_of_circle >= drawn_value:
                    result.append(i)
                    break;
        if len(result) == number_to_generate:
            break;
    return result